## Set Up

In [ ]:
import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer
import pandas as pd
import numpy as np
import requests


gis = GIS()
# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# in memory output file path
wk_memory = "memory" + "\\"


# set workspace and sde connections 
#working_folder = "C:\GIS"
#working_folder = "F:/GIS/GIS_DATA/Monitoring/"
#workspace      = "C:\GIS\Scratch.gdb"
#arcpy.env.workspace = "C:\GIS\Scratch.gdb"
#workspace ='F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
#arcpy.env.workspace = 'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'

# network path to connection files
filePath = "F:\\GIS\\DB_CONNECT"

#path to GDB's to update and master data
master_path = "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb"
SEZ_Master = os.path.join(master_path, "AssessmentUnits_Master")
#set workspace for connection to GDB
workspace=master_path
# database file paths 
### SDE Collection New data collected is put into SDE.Survey under the indicator name
### SDE Vector is where the data will go 
#sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeBase    = os.path.join(filePath, "SarahVector.sde")
#sdeCollect = os.path.join(filepath, "SarahCollect.sde")

# setup connection string???
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

#with engine.begin() as sdeConnect:
 #   erosiondf      = pd.read_sql("SELECT * FROM sde.SDE.Stream_Erosion", sdeConnect)

# local variables sdata is starting data and f data is finishing datatables
ffdata = os.path.join(sdeBase, "sde.SDE.Monitoring")
sdata = os.path.join(sdeCollect, "sde.SDE.Survey")
fdata = os.path.join(master_path, "AssessmentUnits_Master")

##Tables we get the data from in Collect
sezsurveytable = os.path.join(sdata, "sde.SDE.sez_survey")
erosiondata = os.path.join(sdata, "sde.SDE.Stream_Erosion")
incisiondata = os.path.join(sdata, "sde.SDE.sez_channel_incision")
invasivedata = os.path.join(sdata, "sde.SDE.sez_invasive_plant")
headcutdata = os.path.join(sdata, "sde.SDE.sez_stream_headcut")
streamdata = os.path.join(ffdata, "sde.SDE.Stream")


#Staging Tables currently living in SEZ_Data.GDB
stage_bank_stability = os.path.join(master_path, "bank_stability") 
stage_All_SEZ_Scores = os.path.join(master_path, "All_SEZ_Scores")
stage_biotic_integrity = os.path.join(master_path, "biotic_integrity")
stage_headcuts_table = os.path.join(master_path, "heacuts_table")
stage_incision = os.path.join(master_path, "incision")
stage_invasives = os.path.join(master_path, "invasives")



#Final table to append to
#finalSEZtable = os.path.join(fdata, "sde.SDE.SEZ_Assessment_Unit")
#finalSEZtable = os.path.join(master_path, "AssessmentUnits_Master")


# network path to connection files??????
#filePath = "C:\\GIS\\DB_CONNECT"
# network path to connection files
#filePath = r'F:\Research and Analysis\Workspace\Sarah'

#--------------------------------------------#
#Notes to self
#--------------------------------------------#
#F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data\SEZ_Data.gdb. 
###'Assessment_Unit_Master' has all data for SEZ
###“Bioassessment Sample Locations” has stream sites in SEZs




## GDB with Raw Data straight from S123 not in the original folder (that one is not edited)
#headcut23gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2023.gdb")
#headcut22gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2022.gdb")
#headcut20gdb = os.path.join(working_folder,"Stream_Headcut","Stream_Headcut_Survey", "Stream_Headcut_Survey_2020.gdb")
#erosion23gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2023.gdb")
#erosion22gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2022.gdb")
#erosion20gdb = os.path.join(working_folder, "Stream_Erosion","Stream_Erosion_Survey", "Stream_Erosion_Survey_2020.gdb")
#channelincision23gdb = os.path.join(working_folder,"Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2023.gdb")
#channelincision22gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2022.gdb")
#channelincision20gdb = os.path.join(working_folder, "Channel_Incision","Channel_Incision_Survey","Channel_Incision_Survey_2020.gdb")
#invasiveplant23gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2023.gdb")
#invasiveplant22gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2022.gdb")
#invasiveplant20gdb= os.path.join(working_folder,"Invasive_Species","Invasive_Species_Survey","Invasive_Species_Survey_2020.gdb")

#This is thelocatoin for the final SEZ GDB to be updated in the gdb on f drive in the AssessmentUnits Master (polygon) i believe
#FinalGDBtoupdate:F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Data \SEZ_Data.gdb

#Monitoring Dashboard location
#Finalsdelocation:f'Vector.SDE' Sde.Monitoring Sde. SEZ_Assessment_Unit

#Threshold Location? sde.tabular

## Lookup Dictionary for SEZ ID's


In [ ]:
#-----------------------------------------------------------------------------------#
#Create look up dictionary for SEZ_ID fill in--currently works for stream erosion code
#-----------------------------------------------------------------------------------#
#Add in SEZ_Survey table to match up parentglobalid to assessment unit name so we can match an sez_id
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service
# Set workspace environment
arcpy.env.workspace = master_path

# Specify the feature class name SEZ_Master
#feature_class = "AssessmentUnits_Master"
# Create a cursor to iterate over the rows in the feature class
fields = ['Assessment_Unit_Name', 'SEZ_ID']

data = []


# Iterate over the rows in the second feature class and append to data list
with arcpy.da.SearchCursor(SEZ_Master, fields) as cursor:
    for row in cursor:
        data.append(row)
# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data, columns=fields)

# Define function to update SEZ ID lookup dictionary
def update_SEZID_lookup_dict(df, lookup_dict):
    for index, row in df.iterrows():
        # Update SEZ_ID column in DataFrame with data from the lookup dictionary
        df.at[index, 'SEZ_ID'] = lookup_dict.get(row['Assessment_Unit_Name'], None)

    return df

# Assuming 'Assessment_Unit_Name' is the common identifier between DataFrame and lookup dictionary
selected_columns = ['Assessment_Unit_Name', 'SEZ_ID']

# Drop duplicates based on 'Assessment_Unit_Name' and keep the first occurrence
unique_values = df.drop_duplicates(subset='Assessment_Unit_Name', keep='first')

# Convert selected columns to dictionary
lookup_dict = unique_values[selected_columns].set_index('Assessment_Unit_Name').to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

# Update 'SEZ_ID' in other DataFrame with data from the lookup dictionary do this for each indicator...
#Other_df = update_SEZID_lookup_dict(Other_df, lookup_dict)


In [ ]:
#-----------------------------------------------------------------------------------#
# Create look up dictionary for SEZ_ID fill in with Global ID,SEZ ID, and Assessment Unit Name

# Set workspace environment
arcpy.env.workspace = master_path

# Define the fields for SEZ_Master and the additional feature class
Masterfields = ['Assessment_Unit_Name', 'SEZ_ID']
surveyfields = ['assessment_unit_name', 'GlobalID']  # Add any additional fields you need

# Create empty lists to store data
Masterdata = []
surveydata = []

# Iterate over the rows in the SEZ_Master feature class and append to data list
with arcpy.da.SearchCursor(SEZ_Master, Masterfields) as cursor:
    for row in cursor:
        Masterdata.append(row)

# Iterate over the rows in the additional feature class and append to data list
with arcpy.da.SearchCursor(sezsurveytable, surveyfields) as cursor:
    for row in cursor:
        surveydata.append(row)

# Convert the data to Pandas DataFrames
Masterdf = pd.DataFrame(Masterdata, columns=Masterfields)
surveydf = pd.DataFrame(surveydata, columns=surveyfields)
# Convert the column name in surveydf to match Masterdf
surveydf = surveydf.rename(columns={'assessment_unit_name': 'Assessment_Unit_Name'})
# Merge the DataFrames on 'Assessment_Unit_Name'
SEZIDdf = pd.merge(Masterdf, surveydf, on='Assessment_Unit_Name', how='inner')

# Check for missing values in 'GlobalID' or 'SEZ_ID'
missing_values = SEZIDdf[SEZIDdf['GlobalID'].isna() | SEZIDdf['SEZ_ID'].isna()]

# Replace missing values in 'SEZ_ID' with NaN
SEZIDdf.loc[missing_values.index, 'SEZ_ID'] = np.nan



# Display the merged DataFrame
print(SEZIDdf)
# Assuming 'Assessment_Unit_Name' is the common identifier between DataFrame and lookup dictionary
selected_columns = ['Assessment_Unit_Name', 'SEZ_ID', 'GlobalID']

# Drop duplicates based on 'Assessment_Unit_Name' and keep the first occurrence
unique_values = SEZIDdf.drop_duplicates(subset='Assessment_Unit_Name', keep='first')

# Convert selected columns to dictionary
lookup_dict = unique_values[selected_columns].set_index('Assessment_Unit_Name').to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

In [ ]:
import pprint

# Display the dictionary using pprint
pprint.pprint(lookup_dict)


### Code for Grading each parameter

In [ ]:
# Grading for each parameter add to final SDE file?
#Defining Grade for Bank Stability based on Erosiondf[percent_unstable]
def categorize_erosion(Percent_Unstable):
    if 0 <= Percent_Unstable < 5:
        return 'A'
    elif 5 <= Percent_Unstable < 20:
        return 'B'
    elif 20 <= Percent_Unstable < 50:
        return 'C'
    else:
        return 'D'
    
#Scoring based off of grading - check this
def score_indicator(Rating):
    if  Rating == 'A':
        return '12'
    elif Rating == 'B':
        return '9'
    elif Rating == 'C':
        return '6'
    else:
        return '3'

#Define Grade for Incision based off of incisino ratio

def categorize_incision(bankfull_ratio):
    if 0 <= bankfull_ratio < 1.2:
        return 'A'
    elif 1.2 <= bankfull_ratio < 1.6:
        return 'B'
    elif 1.6 <= bankfull_ratio < 2.1:
        return 'C'
    else:
        return 'D'

#Define Grade for Bioassessment Score
#def categorize_csci(biotic_integrity)
 #    if   biotic_integrity > 0.92:
  #      return 'A'
   # elif 0.79 < biotic_integrity <= 0.92:
    #    return 'B'
    #elif 0.62 < biotic_integrity <= 0.79:
     #   return 'C'
    #else:
     #   return 'D'

#Define Grade for Invasive Plant Species Rating

#def categorize_invasive(Invasive_Level)
      




#Define Size for Headcut based off of headcut size---fix this
##A = 0 headcut, B 1+small headcut
#def categorize_headcut(value)
 #    if 0.1 <= value < 0.5:
  #      return 'small'
   # elif 0.5 <= value < 1:
    #    return 'medium'
    #else:
     #   return 'large'

#Define Size for Headcut based off of headcut size---fix this
##A = 0 headcut, B 1+small headcut
#def categorize_headcutgrade(headcutgrade)
 #    if value = 0
  #      return 'A'
   # elif 1.2 <= value < 1.6:
    #    return 'B'
    #elif value =
     #   return 'C'
    #else:
     #   return 'D'

#Define Grade for Invasive 
#A=0, B=1 level 2, C 2 level 2 or 1 level 1, D=3+ level 2 or 2+ level 1


### Stream Erosion 

In [ ]:
#----------------------------------------------------------------#
#Get Data
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service
erosionfields = ['Assessment_Unit_Name', 'Shape.STLength()', 'Bank_Type', 'Percent_Unstable', 'Survey_Date']
#erosiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(erosiondata, erosionfields), columns=erosionfields)

# Initialize an empty list to store data
data = []

 # Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(erosiondata, erosionfields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the data to a Pandas DataFrame
erosiondf = pd.DataFrame(data, columns=erosionfields)

# Replace NaN values in 'Assessment_Unit_Name' column with 'Skylandia SEZ'
erosiondf['Assessment_Unit_Name'] = erosiondf['Assessment_Unit_Name'].fillna('Skylandia SEZ')
# Replace specific values in 'Assessment_Unit_Name' column
erosiondf['Assessment_Unit_Name'] = erosiondf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh'})

#Add SEZ_ID column using lookup dictionary
erosiondf['SEZ_ID'] = erosiondf['Assessment_Unit_Name'].map(lambda x: lookup_dict.get(x, {}).get('SEZ_ID', None))

# Fill NaN values with a specific value, such as 0
erosiondf['SEZ_ID'] = erosiondf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
erosiondf['SEZ_ID'] = erosiondf['SEZ_ID'].astype(int)

#erosiondf = erosiondf.merge(df, on='Assessment_Unit_Name', how='left')

#calculate year column 
erosiondf['Year'] = erosiondf['Survey_Date'].dt.year

# Replace 'both_banks' with 'Both Banks' in Bank_Type column
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['both_banks', 'Both banks'], 'Both Banks' )
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['one_bank', 'One bank'], 'One Bank')
erosiondf['Bank_Type'] = erosiondf['Bank_Type'].replace(['no_bank', 'No bank'], 'No Bank')

#----------------------------------------------------------------#
#Process Data
#----------------------------------------------------------------#

# Initialize variables
erosiondf['bank_multiplier'] = erosiondf['Bank_Type'].apply(lambda x: 2 if x == 'Both Banks' else (1 if x == 'One Bank' else 0))


# Calculate the product of 'Shape.STLength()' and 'bank_multiplier' to get the eroded banks per row
erosiondf['eroded_banks_per_row'] = erosiondf['Shape.STLength()'] * erosiondf['bank_multiplier']

# Group by Assessment_Unit_Name and year and sum the lengths of banks for each unit to get total banks assessed
erosiondf['banks_assessed_per_unit'] = erosiondf.groupby(['Assessment_Unit_Name', 'Year'])['Shape.STLength()'].transform('sum') * 2

# Group by Assessment_Unit_Name and sum the eroded banks per row for each unit
erosiondf['SEZ_total_eroded'] = erosiondf.groupby(['Assessment_Unit_Name', 'Year'])['eroded_banks_per_row'].transform('sum')

# Calculate percent unstable Bank Stability per Assessment Unit
erosiondf['Bank_Stability_Percent_Unstable'] = (erosiondf['SEZ_total_eroded'] / erosiondf['banks_assessed_per_unit']) * 100

#----------------------------------------------------------------#
#Grade, Score
#----------------------------------------------------------------#
erosiondf['Bank_Stability_Rating']=erosiondf['Bank_Stability_Percent_Unstable'].apply(categorize_erosion)
erosiondf['Bank_Stability_Score']= erosiondf['Bank_Stability_Rating'].apply(score_indicator)

erosiondf['Bank_Stability_Data_Source'] = 'TRPA'

erosiondf.head()

print(erosiondf)
#----------------------------------------------------------------#
#post ending dataframe to bank_stability called stage_bank_stability GDB location
#----------------------------------------------------------------#
# Define the name of the feature class
#feature_class_name = 'bank_stability'

# Define the full path to the feature class
#feature_class_path = stage_bank_stability 

#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Bank_Stability_Data_Source': 'Bank_Stability_Data_Source',
    'Bank_Stability_Percent_Unstable': 'Bank_Stability_Percent_Unstable',
    'Bank_Stability_Rating': 'Bank_Stability_Rating',
    'Bank_Stability_Score': 'Bank_Stability_Score',
    'SEZ_ID': 'SEZ_ID'
}

# Rename fields based on field mappings
bank_stabilitydf = erosiondf.rename(columns=field_mapping).drop(columns=[col for col in erosiondf.columns if col not in field_mapping])

readydf = bank_stabilitydf.groupby(['SEZ_ID', 'Year']).first().reset_index()

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

print(readydf)

#Write dataframe to fc in SEZ_Data.GDB
# Convert DataFrame to a list of dictionaries
data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(readydf.columns)

# Append data to existing table
with arcpy.da.InsertCursor(stage_bank_stability, field_names) as cursor:
    for row in data:
        cursor.insertRow([row[field] for field in field_names])

print("Data appended to table successfully.")


#Write dataframe to sde.collect.bank.stability eventually, current code write it to GDB in SEZ_Data.GDB
# Set environment workspace to your SDE connection file
#arcpy.env.workspace = master_path

# Convert DataFrame to Feature Class
#output_feature_class = "ErosionUpdate"  # Name for the output feature class
#output_fc_path = os.path.join(arcpy.env.workspace, output_feature_class)

# Assuming your DataFrame is already converted to a feature class
# Replace "path_to_your_feature_class" with the actual path to your feature class
#arcpy.conversion.TableToTable("path_to_your_feature_class", arcpy.env.workspace, output_feature_class)

# Overwrite Feature Class in SDE
# Replace "path_to_your_dataframe" with the actual path to your DataFrame
#arcpy.management.CopyFeatures("path_to_your_dataframe", output_fc_path)


#print(ready_df)



In [ ]:
print(readydf.dtypes)

In [ ]:
# Convert DataFrame to CSV
#csv_path = os.path.join(working_folder, "Erosiondatamaster.csv")
#ready_df.to_csv(csv_path, index=False) 

In [ ]:
#Create lookup dictionary for parentglobalid,sez_id,Assessment unit

#parentglobalid= global id in sez survey and then we can match upa ssessment unit name... do we even need this 

### Channel Incision

In [ ]:
#----------------------------------------------------------------#
#Get Data
#----------------------------------------------------------------##Convert feature class in gdb to Pandas DataFrame this is for REST Service
incisionfields = ['parentglobalid', 'bankfull_ratio', 'created_date']
#incisiondf = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(erosiondata, erosionfields), columns=erosionfields)

# Initialize an empty list to store data
data = []

 # Iterate over the rows in the feature class and extract data
with arcpy.da.SearchCursor(incisiondata, incisionfields) as cursor:
    for row in cursor:
        data.append(row)

# Convert the data to a Pandas DataFrame
incisiondf = pd.DataFrame(data, columns=incisionfields)

# Replace  values in 'Assessment_Unit_Name' colum
#incisiondf['Assessment_Unit_Name'] = SEZIDdf['Assessment_Unit_Name'].fillna('Skylandia SEZ')
# Replace specific values in 'Assessment_Unit_Name' column
#incisiondf['Assessment_Unit_Name'] = incisiondf['Assessment_Unit_Name'].replace({'Blackwood Creek - upper 2': 'Blackwood Creek - Upper 2', 'Taylor Creek marsh - 1': 'Taylor Creek marsh'})




#----------------------------------------------------------------#
#Add correct info to dataframe
#----------------------------------------------------------------#
# Create a new column 'SEZ ID' initialized with NaN values
incisiondf['GlobalID']= incisiondf['parentglobalid']

# Map ParentGlobalID to SEZ_ID using the lookup dictionary
incisiondf['Assessment_Unit_Name'] = incisiondf['GlobalID'].map(lambda x: lookup_dict[x]['Assessment_Unit_Name'] if x in lookup_dict else np.nan)

#incisiondf['SEZ ID'] = np.nan

# Iterate through the rows in incisiondf
#for index, row in incisiondf.iterrows():
 #   parentglobalid = row['parentglobalid']
    
      # Check if the parent_global_id exists in the lookup dictionary
  #  if parentglobalid in lookup_dict:
        # If it exists, retrieve the corresponding SEZ ID and fill it into SEZ ID column
   #     corresponding_entry = lookup_dict[parentglobalid]
    #    assert row['GlobalID'] == parentglobalid, "ParentGlobalID does not match GlobalID in the lookup dictionary"
     #   incisiondf.at[index, 'SEZ_ID'] = corresponding_entry['SEZ_ID']

# Display the updated incisiondf
print(incisiondf)

#Add SEZ_ID column using lookup dictionary
#incisiondf['SEZ_ID'] = SEZIDdf['GlobalID'].map(lambda x: lookup_dict.get(x, {}).get('SEZ_ID', None))

# Fill NaN values with a specific value, such as 0
#incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].fillna(0)

# Convert SEZ ID column to integer
#incisiondf['SEZ_ID'] = incisiondf['SEZ_ID'].astype(int)

#incisiondf = incisiondf.merge(df, on='Assessment_Unit_Name', how='left')

#calculate year column 
incisiondf['Year'] = incisiondf['created_date'].dt.year


#----------------------------------------------------------------#
#Grade, Score
#----------------------------------------------------------------#
incisiondf['Incision_Rating']=incisiondf['bankfull_ratio'].apply(categorize_incision)
incisiondf['Incision_Score']= incisiondf['Incision_Rating'].apply(score_indicator)

incisiondf['Incision_Data_Source'] = 'TRPA'

incisiondf.head()

print(incisiondf)
#----------------------------------------------------------------#
#post ending dataframe to incision called stage_incision GDB location
#----------------------------------------------------------------#
# Define the name of the feature class
feature_class_name = 'incision'

# Define the full path to the feature class
feature_class_path = stage_incision 

#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Year': 'Year',
    'Incision_Data_Source': 'Incision_Data_Source',
    'Incision_Ratio': 'bankfull_ratio',
    'Incision_Rating': 'Incision_Rating',
    'Incision_Score': 'Incision_Score',
    'SEZ_ID': 'SEZ_ID'
}

# Rename fields based on field mappings
#bank_stabilitydf = incisiondf.rename(columns=field_mapping).drop(columns=[col for col in incisiondf.columns if col not in field_mapping])

#readydf = bank_stabilitydf.groupby(['SEZ_ID', 'Year']).first().reset_index()

# Convert "Year" column to datetime format with year frequency
#readydf['Year'] = pd.to_datetime(readydf['Year'], format='%Y')

#print(readydf)

#Write dataframe to fc in SEZ_Data.GDB
# Convert DataFrame to a list of dictionaries
#data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
#field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(stage_bank_stability, field_names) as cursor:
 ##   for row in data:
   #     cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")

### Invasive Species

In [ ]:
#Invasive Species

### HEadcuts


In [ ]:
#Headcuts

### Bioassessment/ Biotic Integrity

In [ ]:
#Bioassessment scores- get all stream data into sde.Stream first... then look at Biotic Integrit Data Source to find which stream was used to evaluate each meadow.. this will help with percent of stream miles I believe

In [ ]:
#Conifer Encroachment

In [ ]:
#Aquatic Organism /Fish Passage

##### Code to make it a spatially enabled dataframe

In [ ]:
#code to conver to spatially enable geodataframe
# Create a new DataFrame Final_df with columns based on the field mapping
#df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
#sedf = GeoAccessor.from_xy(df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns
#sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise_23_Staging'), sanitize_columns=False)